In [1]:
from graph import *

In [2]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [3]:
g = graph_from_spreadsheet("test_system.xlsx", functions={})

In [4]:
g.nodes

NodeView(('Detector', 'Sensors', 'Level1', 'Level2', 'Level3', 'Disk'))

In [5]:
g.graph

{'globals':    Year
 0  1992,
 'Root Node': 'Disk'}

In [6]:
g.nodes["Detector"]

{'sample data': 1000000,
 'sample rate': 1000,
 'type': 'detector',
 'op efficiency': 0,
 'classifier': <statistics.DummyClassifier at 0x16deeb430>,
 'error matrix': array([[0., 0.],
        [1., 1.]]),
 'rejection ratio': 1.0,
 'data reduction': 1.0,
 'complexity': <function graph.detectors.<locals>.<lambda>(x)>,
 'global ratio': 64.0,
 'message size': 1000000,
 'ops': 1000000,
 'contingency': array([[  0.   ,   0.   ],
        [984.375,  15.625]]),
 'discards': array([0., 0.])}

In [7]:
g.nodes["Sensors"]

{'classifier properties': [1, 0, 1],
 'type': 'processor',
 'rejection ratio': 1,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 64,
 'message size': 1000000.0,
 'ops': 1000000.0,
 'classifier': <statistics.Classifier at 0x28786aa10>,
 'error matrix': array([[0., 0.],
        [1., 1.]]),
 'contingency': array([[  0.   ,   0.   ],
        [984.375,  15.625]]),
 'discards': array([0., 0.]),
 'output rate': 1000.0}

In [8]:
g.nodes["Level1"]

{'classifier properties': [2, 3, 1],
 'type': 'processor',
 'rejection ratio': 2,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 64,
 'message size': 1000000.0,
 'ops': 1000000.0,
 'classifier': <statistics.Classifier at 0x2878576d0>,
 'error matrix': array([[0.50791364, 0.00144048],
        [0.49208636, 0.99855952]]),
 'contingency': array([[4.99977493e+02, 2.25074883e-02],
        [4.84397507e+02, 1.56024925e+01]]),
 'discards': array([4.99977493e+02, 2.25074883e-02]),
 'output rate': 499.9999999417404}

In [9]:
g.nodes["Level1"]["classifier"].error_matrix

array([[0.50791364, 0.00144048],
       [0.49208636, 0.99855952]])

In [10]:
g.nodes["Level2"]

{'classifier properties': [4, 4, 1],
 'type': 'processor',
 'rejection ratio': 4,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 32,
 'message size': 1000000.0,
 'ops': 1000000.0,
 'classifier': <statistics.Classifier at 0x2878577c0>,
 'error matrix': array([[7.74138823e-01, 5.82213866e-04],
        [2.25861177e-01, 9.99417786e-01]]),
 'contingency': array([[3.74990916e+02, 9.08398748e-03],
        [1.09406591e+02, 1.55934085e+01]]),
 'discards': array([3.74990916e+02, 9.08398748e-03]),
 'output rate': 124.9999997587255}

In [11]:
g.nodes["Level3"]

{'classifier properties': [8, 4, 1],
 'type': 'processor',
 'rejection ratio': 8,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 8,
 'message size': 1000000.0,
 'ops': 1000000.0,
 'classifier': <statistics.Classifier at 0x28786abc0>,
 'error matrix': array([[0.99186002, 0.05508584],
        [0.00813998, 0.94491416]]),
 'contingency': array([[108.51602375,   0.85897602],
        [  0.89056748,  14.73443251]]),
 'discards': array([108.51602375,   0.85897602]),
 'output rate': 15.624999986084179}